<a href="https://colab.research.google.com/github/BayMichal/AI_Python/blob/devel/PyTorch_IDC_Cancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install torchviz

  Created wheel for torchviz: filename=torchviz-0.0.2-py3-none-any.whl size=4151 sha256=ea753ef16fad8e68572a257594dcc281c9de8cc453375e501537b1829c2ee0a6
  Stored in directory: /root/.cache/pip/wheels/04/38/f5/dc4f85c3909051823df49901e72015d2d750bd26b086480ec2
Successfully built torchviz


In [2]:
pip install hiddenlayer

In [3]:
# Import Libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import tarfile
from tqdm.notebook import tqdm
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, classification_report
import seaborn as sns
import pandas as pd
import time
from torchviz import make_dot
import hiddenlayer as hl

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets.utils import download_url
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.transforms as tt
from torch.utils.data import random_split
from torchvision.utils import make_grid
from torchvision import transforms
import torchvision.models as models

project_name='Breast-Cancer'


In [8]:
!mkdir ~/.kaggle #create the .kaggle folder in your root directory



mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [9]:
! cp /kaggle.json ~/.kaggle/
#!echo '<"username":"volatille","key":"b06bccd3d1795d29ff0b071f635f2d79">' > ~/.kaggle/kaggle.json #write kaggle API credentials to kaggle.json
!chmod 600 ~/.kaggle/kaggle.json  # set permissions

In [10]:
!pip install -q kaggle #install the kaggle library

In [16]:
import kaggle

!kaggle datasets download "paultimothymooney/breast-histopathology-images"


100% 3.10G/3.10G [00:51<00:00, 52.8MB/s]



In [19]:
!unzip breast-histopathology-images.zip

Strumieniowane dane wyjściowe obcięte do 5000 ostatnich wierszy.
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1551_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1601_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1651_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1701_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1751_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1801_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1851_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1901_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1951_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y2001_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y2251_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y2301_class0.png  
  inflating

In [ ]:
train_tfms = tt.Compose([tt.ToTensor()])
valid_tfms = tt.Compose([tt.ToTensor()])

In [ ]:
import os
os.chdir("/content/drive/MyDrive/Magisterka")
!ls

test  train  validation


In [ ]:
# Load train, validation and test dataset
path = "/content/drive/MyDrive/Magisterka/"

train_file = os.path.join(path, "train")
val_file = os.path.join(path, "validation")
test_file = os.path.join(path, "test")



In [ ]:
train_file

'/content/drive/MyDrive/Magisterka/train'

In [ ]:
import os

path = train_file
path2 = '/content/drive/MyDrive/Magisterka/train/0'

files = os.listdir(path2)

for f in tqdm(files[0:10]):
	print(f)

OSError: ignored

In [ ]:
train_ds = ImageFolder(train_file, train_tfms)


FileNotFoundError: ignored